<a href="https://colab.research.google.com/github/vinodwij/dsgp-group23-business-data-harbour/blob/uzama-a/Sentiment_Analysis_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
#loading the data from csv file to pandas dataframe
review_data = pd.read_csv('/content/Watch Review dataset.csv', encoding = 'ISO-8859-1')

In [6]:
#checking the number of rows and columns
review_data.shape

(2125, 82)

In [7]:
review_data = review_data[['product_title', 'review_rating', 'review_text']]

In [8]:
#counting the number of missing values in the
review_data.isnull().sum()

product_title    180
review_rating    180
review_text      185
dtype: int64

In [9]:
#removing the rows that contain null values
review_data = review_data.dropna()
review_data

,product_title,review_rating,review_text
0,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,"bought it as a replacement, turns out the wris..."
1,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,CASIO WAS ONCE A TOP OF THE LINE WATCH. DEPEND...
2,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Everything works the way it should but it is tiny
3,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Small. For Kids wrist not adults ..
4,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Did not read the spes. Carefully just satisfie...
...,...,...,...
1941,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,It was smaller than expected
1942,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,These were better made in the 80's. For all th...
1943,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,Light doesnât illuminate very well. Watch ba...
1944,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,The watch certainly keeps very good time but t...


In [10]:
review_data['ID'] =  review_data.index + 1

In [11]:
review_data.head()

,product_title,review_rating,review_text,ID
0,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,"bought it as a replacement, turns out the wris...",1
1,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,CASIO WAS ONCE A TOP OF THE LINE WATCH. DEPEND...,2
2,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Everything works the way it should but it is tiny,3
3,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Small. For Kids wrist not adults ..,4
4,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Did not read the spes. Carefully just satisfie...,5


In [12]:
#counting the number of missing values in the
review_data.isnull().sum()

product_title    0
review_rating    0
review_text      0
ID               0
dtype: int64

In [13]:
#checking the distribution of the target column
review_data['review_rating'].value_counts()

3.0    511
5.0    400
1.0    363
4.0    353
2.0    313
Name: review_rating, dtype: int64

In [14]:
review_points = review_data['review_rating'].tolist()

In [15]:
review_label = []
for i in review_points:
  if i == 1 or i == 2:
    i = 0
  elif i == 3:
    i = 3
  else:
    i = 1
  review_label.append(i)

In [ ]:
review_label

In [17]:
review_data['review_label'] = review_label

In [18]:
review_data

,product_title,review_rating,review_text,ID,review_label
0,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,"bought it as a replacement, turns out the wris...",1,0
1,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,CASIO WAS ONCE A TOP OF THE LINE WATCH. DEPEND...,2,0
2,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Everything works the way it should but it is tiny,3,0
3,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Small. For Kids wrist not adults ..,4,0
4,Casio Men's 'Vintage' Quartz Metal and Resin C...,2.0,Did not read the spes. Carefully just satisfie...,5,0
...,...,...,...,...,...
1941,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,It was smaller than expected,1942,3
1942,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,These were better made in the 80's. For all th...,1943,3
1943,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,Light doesnât illuminate very well. Watch ba...,1944,3
1944,Casio Men's 'Vintage' Quartz Metal and Resin C...,3.0,The watch certainly keeps very good time but t...,1945,3


In [19]:
#checking the distribution of the target column
review_data['review_label'].value_counts()

1    753
0    676
3    511
Name: review_label, dtype: int64